In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb
import eli5

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from eli5.sklearn import PermutationImportance


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [47]:
df = pd.read_hdf('data/car.h5') #wczytanie pliku z danymi
df.shape

(106494, 155)

##Feature Engineering


In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
     df[feat + SUFFIX_CAT] = factorized_values
    

In [50]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x] 
cat_feats = [x for x in cat_feats if 'price' not in x] #wykluczamy kolumny ktore posiadaja CENY czyli liczby - wiec nie robimy faktoryzacji ID 
len(cat_feats)

151

In [51]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X , y , cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19695.13091100928, 148.72570644015792)

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values

  
  scores = cross_val_score(model, X , y , cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


##DecisionTree

In [53]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats)

(-19695.13091100928, 148.72570644015792)

##Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0) #50 drzew, ^ w poprzednim modelu tylko jedno drzewo
run_model(model,cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [55]:
xgb_params = {
    'max_depth': 5, 
    'n_estimators': 50, 
    'learning_rate': 0.1,
    'seed': 0
    
}
model = xgb.XGBRegressor(**xgb_params)
run_model(model,cat_feats)

[20:55:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:56:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [0]:
# sprawdzenie które cechy wg. XBoost są istotne
m = xgb.XGBRegressor(**xgb_params)
m.fit (X,y)

imp = PermutationImportance(m,random_state=0).fit(X,y)
eli5.show_weights(imp,feature_names = cat_feats)

[19:56:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [56]:
newFeats = [
    'param_napęd__cat',
    'param_rok-produkcji__cat',
    'param_stan__cat',
    'param_skrzynia-biegów__cat',
    'param_faktura-vat__cat',
    'param_moc__cat',
    'param_marka-pojazdu__cat',
    'feature_kamera-cofania__cat',
    'param_typ__cat',
    'param_pojemność-skokowa__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'param_model-pojazdu__cat',
    'param_wersja__cat',
    'param_kod-silnika__cat',
    'feature_system-start-stop__cat',
    'feature_asystent-pasa-ruchu__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'feature_łopatki-zmiany-biegów__cat',
    'feature_regulowane-zawieszenie__cat',
]

run_model(model,newFeats)

[20:59:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:59:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:59:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
# optymalizacja wybranych parametrów (domyslnie numerycznych np. jak rok, moc, pojemnosc) ktorych nie trzeba faktoryzowac i nadawac im ID do cech 
# i np. w tym przypadku ciągłośc numeryczna paramteru została zatracona (np. 2018 = 0 a 2011 = 1 )..
df['param_rok-produkcji__cat'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, -1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75])

In [57]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
# rok produkcji
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [60]:
newFeats = [
    'param_napęd__cat',
    'param_rok-produkcji',
    'param_stan__cat',
    'param_skrzynia-biegów__cat',
    'param_faktura-vat__cat',
    'param_moc__cat',
    'param_marka-pojazdu__cat',
    'feature_kamera-cofania__cat',
    'param_typ__cat',
    'param_pojemność-skokowa__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'param_model-pojazdu__cat',
    'param_wersja__cat',
    'param_kod-silnika__cat',
    'feature_system-start-stop__cat',
    'feature_asystent-pasa-ruchu__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'feature_łopatki-zmiany-biegów__cat',
    'feature_regulowane-zawieszenie__cat',
]

run_model(model,newFeats)

[21:03:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:04:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:04:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [0]:
# moc
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [62]:
newFeats = [
    'param_napęd__cat',
    'param_rok-produkcji',
    'param_stan__cat',
    'param_skrzynia-biegów__cat',
    'param_faktura-vat__cat',
    'param_moc',
    'param_marka-pojazdu__cat',
    'feature_kamera-cofania__cat',
    'param_typ__cat',
    'param_pojemność-skokowa__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'param_model-pojazdu__cat',
    'param_wersja__cat',
    'param_kod-silnika__cat',
    'feature_system-start-stop__cat',
    'feature_asystent-pasa-ruchu__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'feature_łopatki-zmiany-biegów__cat',
    'feature_regulowane-zawieszenie__cat',
]

run_model(model,newFeats)

[21:07:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:07:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [63]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
# pojemnosc skokowa
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))


In [65]:
newFeats = [
    'param_napęd__cat',
    'param_rok-produkcji',
    'param_stan__cat',
    'param_skrzynia-biegów__cat',
    'param_faktura-vat__cat',
    'param_moc',
    'param_marka-pojazdu__cat',
    'feature_kamera-cofania__cat',
    'param_typ__cat',
    'param_pojemność-skokowa',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'param_model-pojazdu__cat',
    'param_wersja__cat',
    'param_kod-silnika__cat',
    'feature_system-start-stop__cat',
    'feature_asystent-pasa-ruchu__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'feature_łopatki-zmiany-biegów__cat',
    'feature_regulowane-zawieszenie__cat',
]

run_model(model,newFeats)

[21:13:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:13:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:13:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)